In [12]:
import requests
import web3
import pandas as pd

import libs.common.utils as utils
import libs.aave.api as aave_api
import libs.common.payload as payload
import libs.common.etherscan as es

In [13]:
node_url = "https://stylish-wider-patina.quiknode.pro/274049e810f4dee5f0807cd426196aeafd438838/"

In [14]:
ws = web3.Web3(web3.Web3.HTTPProvider(node_url))

In [15]:
contract_address = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"
function_sig = "coins(uint256)"

In [16]:
def func_call_payload(id, contract_address, function_sig, block_number, values):
    func_hex = utils.get_function_hex(function_sig)
    param_types = utils.get_function_param_types(function_sig)
    assert len(param_types) == len(values)
    data = func_hex
    
    for t, v in zip(param_types,values):
        if type(v) == str: # address
            data += v[2:].zfill(64)
        else:
            data += hex(v)[2:].zfill(64)
    
    payload = {
        "id":0,
        "jsonrpc":"2.0",
        "method":"eth_call",
        "params": [
            {
                "to": contract_address,
                "data": data
            },
            block_number
        ]
    }
    return payload

In [28]:
payload = func_call_payload(0, contract_address, function_sig, hex(18000000), [0])
resp = requests.post(node_url, json=payload)
resp.json()

{'jsonrpc': '2.0',
 'id': 0,
 'result': '0x0000000000000000000000006b175474e89094c44da98b954eedeac495271d0f'}

In [31]:
payload = func_call_payload(0, contract_address, "get_dy(int128,int128,uint256)", hex(18000000), [1,2,100000])
resp = requests.post(node_url, json=payload)
int(resp.json()['result'],16)

100052

In [ ]:
# abi -> param types -> pad